In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score
import torch
from torch import nn

df = pd.read_csv("data/synthetic_em_data.csv")
X = df[["Time", "dBdt"]].values
y = df[["Depth","Resistivity"]].values
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X = scaler_X.fit_transform(X)
y = scaler_y.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [24]:
class EMModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(2, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 2)
        )
    
    def forward(self, x):
        return self.layers(x)

model = EMModel()
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)


In [25]:
X_train_t = torch.tensor(X_train, dtype=torch.float32)
y_train_t = torch.tensor(y_train, dtype=torch.float32)

EPOCHS = 10000
for epoch in range(EPOCHS):
    model.train()
    y_pred = model(X_train_t)
    loss = loss_fn(y_pred, y_train_t)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if epoch % 100 == 0 or epoch == 9999:
        print(f"Epoch {epoch}: Loss = {loss.item():.6f}")


Epoch 0: Loss = 1.031776
Epoch 100: Loss = 1.001517
Epoch 200: Loss = 1.000582
Epoch 300: Loss = 0.999698
Epoch 400: Loss = 0.998958
Epoch 500: Loss = 0.998225
Epoch 600: Loss = 0.997430
Epoch 700: Loss = 0.996786
Epoch 800: Loss = 0.996324
Epoch 900: Loss = 0.995831
Epoch 1000: Loss = 0.995434
Epoch 1100: Loss = 0.995097
Epoch 1200: Loss = 0.994804
Epoch 1300: Loss = 0.994582
Epoch 1400: Loss = 0.994359
Epoch 1500: Loss = 0.994193
Epoch 1600: Loss = 0.994074
Epoch 1700: Loss = 0.993964
Epoch 1800: Loss = 0.993898
Epoch 1900: Loss = 0.993865
Epoch 2000: Loss = 0.993812
Epoch 2100: Loss = 0.993779
Epoch 2200: Loss = 0.993753
Epoch 2300: Loss = 0.993730
Epoch 2400: Loss = 0.993714
Epoch 2500: Loss = 0.993720
Epoch 2600: Loss = 0.993713
Epoch 2700: Loss = 0.993675
Epoch 2800: Loss = 0.993678
Epoch 2900: Loss = 0.993645
Epoch 3000: Loss = 0.993640
Epoch 3100: Loss = 0.993636
Epoch 3200: Loss = 0.993630
Epoch 3300: Loss = 0.993611
Epoch 3400: Loss = 0.993602
Epoch 3500: Loss = 0.993636
Epoc

In [26]:
def predict_dbdt(time, dBdt):
    inp = torch.tensor(scaler_X.transform([[time, dBdt]]), dtype=torch.float32)
    model.eval()
    with torch.no_grad():
        pred = model(inp)
    return scaler_y.inverse_transform(pred.numpy())[0][:2]

print(predict_dbdt(1e-05, -5.60886297079976E-10))

[-194.07242  786.67914]


In [ ]:
# !mkdir models
# torch.save(model.state_dict(), "models/em_model.pth")
# print(f"Model Saved to models/em_model.pth")

Model Saved to models/em_model.pth
